In [ ]:
# placeholder jupyter notebook for the final project

In [ ]:
import os

import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt


# Assign spreadsheet filename to 'file'
Sales_file = 'Louisville Sales.xlsx'

# Load spreadsheet
salesxl = pd.ExcelFile(Sales_file)

# Print the sheet names
print(salesxl.sheet_names)

In [ ]:
# Load a sheet into a DataFrame by name: df1
df1 = salesxl.parse('sheet1')

# have to clean up the Age data because it is not always correct and it is being used in some decision making later

df1['Age'] = np.where(df1['Age'] > 3000, 0, df1['Age'])
df1['Age'] = np.where(df1['Age'] > 1000, 2020 - df1['Age'].astype(int), df1['Age'])
df1['Age'] = np.where(df1['Age'] > 400, 2020 - df1['Year Built'].astype(int), df1['Age'])
df1['Age'] = np.where(df1['Age'] > 400, 0, df1['Age'])



In [ ]:
Rent_file = 'Rental data.xlsx'
rentxl = pd.ExcelFile(Rent_file)
print(rentxl.sheet_names)

In [ ]:
df2 = rentxl.parse('sheet1')

# needed to clean up the Rental Age data as well from wrong inputs

df2['Age'] = np.where(df2['Age'] > 3000, 0, df2['Age'])
df2['Age'] = np.where(df2['Age'] > 1800, 2020 - df2['Age'].astype(int), df2['Age'])
df2['Age'] = np.where(df2['Age'] > 400, 0, df2['Age'])


In [ ]:
# need to drop rental data that is from before 2019 to make sure it is relevant
df2 = df2[(df2['Status Change Date'].dt.year >= 2019)]


In [ ]:
# calculating subdivision level average price per sqft and average age and average size for both rental and sales

groupped_sales_df = df1.groupby('Subdivision/Condo')['Sold Price/SqFt', 'Age', 'SqFt -Total Finished'].mean().rename(columns={
    "Sold Price/SqFt": "Average_Sold_Price", 
    "Age": "Average_Sold_Age", 
    "SqFt -Total Finished": "Average_Sold_SqFt"
})

sales_count = df1.groupby('Subdivision/Condo')['Sold Price/SqFt', 'Age', 'SqFt -Total Finished'].size().reset_index(name='sales_count')
groupped_sales_df = pd.merge(groupped_sales_df, sales_count, on="Subdivision/Condo")
groupped_sales_df.head()

In [ ]:
groupped_rental_df = df2.groupby('Subdivision/Condo')['List Price/SqFt', 'Age', 'SqFt -Total Finished'].mean().rename(columns={
    "List Price/SqFt": "Average_Rent_Price", 
    "Age": "Average_Rent_Age", 
    "SqFt -Total Finished": "Average_Rent_SqFt"
})


rent_count = df2.groupby('Subdivision/Condo')['List Price/SqFt', 'Age', 'SqFt -Total Finished'].size().reset_index(name='rent_count')

groupped_rental_df = pd.merge(groupped_rental_df, rent_count, on="Subdivision/Condo")

groupped_rental_df.head()

In [ ]:
# Merge the 2 dataframes and only keep the subdivisions where there is both sales and rental data available 

subdivision_df = pd.merge(groupped_sales_df, groupped_rental_df, on="Subdivision/Condo", how='inner')
subdivision_df.head()

In [ ]:
subdivision_df['rent_sales_ratio'] = pd.eval('subdivision_df.Average_Rent_Price/subdivision_df.Average_Sold_Price')*1000


In [ ]:
# exclude items from the dataframe where the average SqFt and/or Age of the houses sold are very different (more than 50% higher
# or lower) from the average age and/of SqFt of the rented homes -> the Rental and Sales price comparison is not relevant

subdivision_df['rent_sales_age_percent'] = pd.eval('abs((subdivision_df.Average_Sold_Age-subdivision_df.Average_Rent_Age)/subdivision_df.Average_Sold_Age)')
subdivision_df['rent_sales_SqFt_percent'] = pd.eval('abs((subdivision_df.Average_Sold_SqFt-subdivision_df.Average_Rent_SqFt)/subdivision_df.Average_Sold_SqFt)')
subdivision1_df = subdivision_df.drop(subdivision_df[subdivision_df['rent_sales_age_percent'] > .5].index)
reduced_subdivision_df = subdivision1_df.drop(subdivision1_df[subdivision1_df['rent_sales_SqFt_percent'] > .5].index)
reduced_subdivision_df.head()

In [ ]:
# show the subdivisions with the highest rent(per sqft)/sales(per sqft) ratio -> where you get the most bang for your buck
# if investing in a rental property

sorted_subdivision_df = reduced_subdivision_df.sort_values(['rent_sales_ratio'], ascending=False)
sorted_subdivision_df.head()

*Calculate the same for zip codes*

In [ ]:
# creating the calculated file for the Sales data, same as above, but now the groupping is based on zip code

groupped_zip_sales_df = df1.groupby('Zip Code')['Sold Price/SqFt', 'Age', 'SqFt -Total Finished'].mean().rename(columns={
    "Sold Price/SqFt": "Average_Sold_Price", 
    "Age": "Average_Sold_Age", 
    "SqFt -Total Finished": "Average_Sold_SqFt"
})
sales_count = df1.groupby('Zip Code')['Sold Price/SqFt', 'Age', 'SqFt -Total Finished'].size().reset_index(name='sales_count')
groupped_zip_sales_df = pd.merge(groupped_zip_sales_df, sales_count, on="Zip Code")
groupped_zip_sales_df.head()

In [ ]:
# same groupped, calculated file for the Rental data based on zip code

groupped_zip_rental_df = df2.groupby('Zip Code')['List Price/SqFt', 'Age', 'SqFt -Total Finished'].mean().rename(columns={
    "List Price/SqFt": "Average_Rent_Price", 
    "Age": "Average_Rent_Age", 
    "SqFt -Total Finished": "Average_Rent_SqFt"
})


rent_count = df2.groupby('Zip Code')['List Price/SqFt', 'Age', 'SqFt -Total Finished'].size().reset_index(name='rent_count')

groupped_zip_rental_df = pd.merge(groupped_zip_rental_df, rent_count, on="Zip Code")

groupped_zip_rental_df.head()

In [ ]:
# Merge the 2 dataframes and only keep the Zip Codes where there is both sales and rental data available 

zip_df = pd.merge(groupped_zip_sales_df, groupped_zip_rental_df, on="Zip Code", how='inner')
zip_df.head()

In [ ]:
zip_df['rent_sales_ratio'] = pd.eval('zip_df.Average_Rent_Price/zip_df.Average_Sold_Price')*1000


In [ ]:
# exclude items from the dataframe where the average SqFt and/or Age of the houses sold are very different (more than 50% higher
# or lower) from the average age and/of SqFt of the rented homes -> the Rental and Sales price comparison is not relevant

zip_df['rent_sales_age_percent'] = pd.eval('abs((zip_df.Average_Sold_Age-zip_df.Average_Rent_Age)/zip_df.Average_Sold_Age)')
zip_df['rent_sales_SqFt_percent'] = pd.eval('abs((zip_df.Average_Sold_SqFt-zip_df.Average_Rent_SqFt)/zip_df.Average_Sold_SqFt)')
zip1_df = zip_df.drop(zip_df[zip_df['rent_sales_age_percent'] > .5].index)
reduced_zip_df = zip1_df.drop(zip1_df[zip1_df['rent_sales_SqFt_percent'] > .5].index)
reduced_zip_df.head()

In [ ]:
# show the subdivisions with the highest rent(per sqft)/sales(per sqft) ratio -> where you get the most bang for your buck
# if investing in a rental property

sorted_zip_df = reduced_zip_df.sort_values(['rent_sales_ratio'], ascending=False)
sorted_zip_df.head()

We have selected the top zip codes to invest in based on average Sales price and Rental income

Next, in the original dataframes I will create tiers based on SqFt to be able to demonstrate the connection between relative house price to monthly rental income for houses of various sizes in the same zip code (what is the cost of entry and the expected rental income per month)

1. I will create tiers based on size
2. I will add a new column to each original dataframes that adds the tiers for each entry
3. I will group them by zip code and by size tier
4. I will merge the 2 databases
5. I will visualize the outcome for the top 5 zip codes


In [ ]:
conditions_Sales = [
    (df1['SqFt -Total Finished'] <= 1000),
    (df1['SqFt -Total Finished'] > 1000) & (df1['SqFt -Total Finished'] <= 2000),
    (df1['SqFt -Total Finished'] > 2000) & (df1['SqFt -Total Finished'] <= 3000),
    (df1['SqFt -Total Finished'] > 3000) & (df1['SqFt -Total Finished'] <= 4000),
    (df1['SqFt -Total Finished'] > 4000)
]

conditions_Rental = [
    (df2['SqFt -Total Finished'] <= 1000),
    (df2['SqFt -Total Finished'] > 1000) & (df2['SqFt -Total Finished'] <= 2000),
    (df2['SqFt -Total Finished'] > 2000) & (df2['SqFt -Total Finished'] <= 3000),
    (df2['SqFt -Total Finished'] > 3000) & (df2['SqFt -Total Finished'] <= 4000),
    (df2['SqFt -Total Finished'] > 4000)
]

values = ['0 - 1000', '1000 - 2000', '2000 - 3000', '3000 - 4000', '4000+',]

df1['size group'] = np.select(conditions_Sales, values)
df2['size group'] = np.select(conditions_Rental, values)

df1.head()

In [ ]:
groupped_zip_sales_size_df = df1.groupby(['Zip Code', 'size group'])[['Sold Price', 'Age', 'SqFt -Total Finished']].mean().rename(columns={
    "Sold Price": "Average_Sold_Price", 
    "Age": "Average_Sold_Age", 
    "SqFt -Total Finished": "Average_Sold_SqFt"
})
sales_count = df1.groupby(['Zip Code', 'size group'])[['Sold Price', 'Age', 'SqFt -Total Finished']].size().reset_index(name='sales_count')
groupped_zip_sales_size_df = pd.merge(groupped_zip_sales_size_df, sales_count, on=['Zip Code', 'size group'])
# format the numbers in the table
pd.options.display.float_format = '{:,.0f}'.format
groupped_zip_sales_size_df.head(10)

In [ ]:
groupped_zip_rental_size_df = df2.groupby(['Zip Code', 'size group'])[['Rent/Month', 'Age', 'SqFt -Total Finished']].mean().rename(columns={
    "Rent/Month": "Average_Monthly_Rent", 
    "Age": "Average_Sold_Age", 
    "SqFt -Total Finished": "Average_Sold_SqFt"
})
sales_count = df2.groupby(['Zip Code', 'size group'])[['Rent/Month', 'Age', 'SqFt -Total Finished']].size().reset_index(name='sales_count')
groupped_zip_rental_size_df = pd.merge(groupped_zip_rental_size_df, sales_count, on=['Zip Code', 'size group'])
# format the numbers in the table
pd.options.display.float_format = '{:,.0f}'.format
groupped_zip_rental_size_df.head()

In [ ]:
zip_size_group_df = pd.merge(groupped_zip_sales_size_df, groupped_zip_rental_size_df, on=['Zip Code', 'size group'], how='inner')
zip_size_group_df.head()

In [ ]:
sorted_zip_df = sorted_zip_df.reset_index(drop=True)
sorted_zip_df.head()

In [ ]:
top_5_results = sorted_zip_df.loc[0:4, 'Zip Code']
print(top_5_results)

In [ ]:
reduced_df = pd.DataFrame(zip_size_group_df, columns = ['Zip Code', 'size group', 'Average_Sold_Price', 'Average_Monthly_Rent']) 
reduced_df
best_value = top_5_results.iloc[0]
best_results_df = pd.DataFrame(reduced_df[reduced_df['Zip Code'] == best_value], columns = ['size group', 'Average_Sold_Price', 'Average_Monthly_Rent'])
best_results_df

In [ ]:
second_best_value = top_5_results.iloc[1]
second_best_results_df = pd.DataFrame(reduced_df[reduced_df['Zip Code'] == second_best_value], columns = ['size group', 'Average_Sold_Price', 'Average_Monthly_Rent'])
third_best_value = top_5_results.iloc[2]
third_best_results_df = pd.DataFrame(reduced_df[reduced_df['Zip Code'] == third_best_value], columns = ['size group', 'Average_Sold_Price', 'Average_Monthly_Rent'])
fourth_best_value = top_5_results.iloc[3]
fourth_best_results_df = pd.DataFrame(reduced_df[reduced_df['Zip Code'] == fourth_best_value], columns = ['size group', 'Average_Sold_Price', 'Average_Monthly_Rent'])
fifth_best_value = top_5_results.iloc[4]
fifth_best_results_df = pd.DataFrame(reduced_df[reduced_df['Zip Code'] == fifth_best_value], columns = ['size group', 'Average_Sold_Price', 'Average_Monthly_Rent'])


In [ ]:
#create a line chart with a secondary y axis to display both the Sales and rental price by size group for the particular zip code

fig = plt.figure()
ax1 = fig.add_subplot(111)
ax2 = ax1.twinx()

# the ax keyword sets the axis that the data frame plots to
second_best_results_df.plot(ax=ax1, x='size group', y='Average_Sold_Price')
second_best_results_df.plot(ax=ax2, x='size group', y='Average_Monthly_Rent', color='g')
ax1.set_xlabel('size group')
ax1.set_ylabel('Average_Sold_Price')
ax2.set_ylabel('Average_Monthly_Rent')
ax1.legend(loc=0)
ax2.legend(loc='lower right')
fig.set_figwidth(10)
ax1.set_title(f'Sales price and rental income by size in {second_best_value}')

plt.show()
second_best_results_df

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)
ax2 = ax1.twinx()

# the ax keyword sets the axis that the data frame plots to
third_best_results_df.plot(ax=ax1, x='size group', y='Average_Sold_Price')
third_best_results_df.plot(ax=ax2, x='size group', y='Average_Monthly_Rent', color='g')
ax1.set_xlabel('size group')
ax1.set_ylabel('Average_Sold_Price')
ax2.set_ylabel('Average_Monthly_Rent')
ax1.legend(loc=0)
ax2.legend(loc='lower right')
fig.set_figwidth(10)
ax1.set_title(f'Sales price and rental income by size in {third_best_value}')

plt.show()
third_best_results_df

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)
ax2 = ax1.twinx()

# the ax keyword sets the axis that the data frame plots to
fourth_best_results_df.plot(ax=ax1, x='size group', y='Average_Sold_Price')
fourth_best_results_df.plot(ax=ax2, x='size group', y='Average_Monthly_Rent', color='g')
ax1.set_xlabel('size group')
ax1.set_ylabel('Average_Sold_Price')
ax2.set_ylabel('Average_Monthly_Rent')
ax1.legend(loc=0)
ax2.legend(loc='lower right')
fig.set_figwidth(10)
ax1.set_title(f'Sales price and rental income by size in {fourth_best_value}')

plt.show()
fourth_best_results_df

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)
ax2 = ax1.twinx()

# the ax keyword sets the axis that the data frame plots to
fifth_best_results_df.plot(ax=ax1, x='size group', y='Average_Sold_Price')
fifth_best_results_df.plot(ax=ax2, x='size group', y='Average_Monthly_Rent', color='g')
ax1.set_xlabel('size group')
ax1.set_ylabel('Average_Sold_Price')
ax2.set_ylabel('Average_Monthly_Rent')
ax1.legend(loc=0)
ax2.legend(loc='lower right')
fig.set_figwidth(10)
ax1.set_title(f'Sales price and rental income by size in {fifth_best_value}')

plt.show()
fifth_best_results_df